In [3]:
# !pip install datasets

In [4]:
# %%
# 1. Imports
import pandas as pd
from datasets import load_dataset
from cryptography.fernet import Fernet
import hashlib
import os

In [5]:
# 2. Charger le dataset IMDb
dataset = load_dataset("imdb")
df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# 3. Générer une clé de chiffrement (à stocker)
key = Fernet.generate_key()
fernet = Fernet(key)

In [7]:
# 4. Créer un hash pour simuler un identifiant utilisateur
df['user_id'] = ['user' + str(i) for i in range(len(df))]
df['user_hash'] = df['user_id'].apply(lambda uid: hashlib.sha256(uid.encode()).hexdigest())

In [8]:
# 5. Chiffrer les messages
df['encrypted_text'] = df['text'].apply(lambda txt: fernet.encrypt(txt.encode()).decode())

In [9]:
# 6. Garder seulement ce qui est utile
df_secure = df[['user_hash', 'encrypted_text', 'label']]
os.makedirs("data", exist_ok=True)
df_secure.to_csv("data/encrypted_imdb.csv", index=False)

print("✅ Données chiffrées sauvegardées dans 'data/encrypted_imdb.csv'")

✅ Données chiffrées sauvegardées dans 'data/encrypted_imdb.csv'


In [10]:
# %%
# 7. Sauvegarder la clé
with open("data/fernet.key", "wb") as f:
    f.write(key)
print("🔐 Clé Fernet sauvegardée dans 'data/fernet.key'")

🔐 Clé Fernet sauvegardée dans 'data/fernet.key'


In [11]:
# %%
# 8. Exemple : recharger la clé et déchiffrer un message
with open("data/fernet.key", "rb") as f:
    key = f.read()

fernet = Fernet(key)

In [12]:
# Exemple de déchiffrement
example_encrypted = df_secure['encrypted_text'].iloc[0]
decrypted = fernet.decrypt(example_encrypted.encode()).decode()
print("🔓 Exemple déchiffré :", decrypted)

🔓 Exemple déchiffré : I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are fe